In [109]:
from langchain.agents import AgentExecutor, ZeroShotAgent, create_openai_tools_agent
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_openai import OpenAI
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage



load_dotenv('../../.env')

True

In [110]:
from langchain_experimental.tools import PythonREPLTool

tools = [PythonREPLTool()]

In [111]:
# prompt = hub.pull("hwchase17/openai-tools-agent")
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You are a helpful GIS agent/consultant.'),
        MessagesPlaceholder(variable_name='chat_history'),
        HumanMessagePromptTemplate.from_template('{input}'),
        MessagesPlaceholder(variable_name='agent_scratchpad')
    ]
)
memory = ConversationBufferWindowMemory(k=6,
    memory_key="chat_history", return_messages=True, input_key='input', output_key='output')

In [112]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessage(content='You are a helpful GIS agent/consultant.'), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(variable_name='agent_scratchpad')])

In [113]:
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)
agent = create_openai_tools_agent(
    llm=llm, tools=tools, prompt=prompt)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory, #input_variables=['agent_scratchpad', 'chat_history', 'input'], agent_executor_kwargs={"memory": memory}
)

In [114]:
for chunk in agent_executor.stream({"input": "what is the weather in SF and then LA"}):
    print(chunk)
    print("------")



> Entering new AgentExecutor chain...


{'actions': [OpenAIToolAgentAction(tool='Python_REPL', tool_input="import requests\n\n# Get weather for San Francisco\nsf_weather = requests.get('http://api.openweathermap.org/data/2.5/weather?q=San%20Francisco&appid=YOUR_API_KEY')\nprint(sf_weather.json())", log="\nInvoking: `Python_REPL` with `import requests\n\n# Get weather for San Francisco\nsf_weather = requests.get('http://api.openweathermap.org/data/2.5/weather?q=San%20Francisco&appid=YOUR_API_KEY')\nprint(sf_weather.json())`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_9eHvkpg3guQDZcwc2BwO9tat', 'function': {'arguments': '{"__arg1": "import requests\\n\\n# Get weather for San Francisco\\nsf_weather = requests.get(\'http://api.openweathermap.org/data/2.5/weather?q=San%20Francisco&appid=YOUR_API_KEY\')\\nprint(sf_weather.json())"}', 'name': 'Python_REPL'}, 'type': 'function'}, {'index': 1, 'id': 'call_lwr7KRUtxxgJbk2UNRCe4XHQ', 'function': {'arguments': '{"__arg1": "import

In [115]:
agent_executor.invoke({'input': "What is 2 + 2?", 'chat_history': []})



> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `2 + 2`


The result of 2 + 2 is 4.

> Finished chain.


{'input': 'What is 2 + 2?',
 'chat_history': [HumanMessage(content='what is the weather in SF and then LA'),
  AIMessage(content='It seems that I need to provide a valid API key to retrieve the weather information. I will need to obtain a valid API key in order to proceed with fetching the weather data for San Francisco and Los Angeles.')],
 'output': 'The result of 2 + 2 is 4.'}

In [116]:
memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='what is the weather in SF and then LA'), AIMessage(content='It seems that I need to provide a valid API key to retrieve the weather information. I will need to obtain a valid API key in order to proceed with fetching the weather data for San Francisco and Los Angeles.'), HumanMessage(content='What is 2 + 2?'), AIMessage(content='The result of 2 + 2 is 4.')]), output_key='output', input_key='input', return_messages=True, memory_key='chat_history', k=6)

In [117]:
agent_executor.invoke({'input': "What is the double of that?", 'chat_history': []})



> Entering new AgentExecutor chain...



Invoking: `Python_REPL` with `2 * 2`


The double of 4 is 8.

> Finished chain.


{'input': 'What is the double of that?',
 'chat_history': [HumanMessage(content='what is the weather in SF and then LA'),
  AIMessage(content='It seems that I need to provide a valid API key to retrieve the weather information. I will need to obtain a valid API key in order to proceed with fetching the weather data for San Francisco and Los Angeles.'),
  HumanMessage(content='What is 2 + 2?'),
  AIMessage(content='The result of 2 + 2 is 4.')],
 'output': 'The double of 4 is 8.'}